In [1]:
import re, os
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
import sqlite3
import time

data_path = 'C:/Users/SpiffyApple/Documents/USC/OwnResearch/marketShare'
dbname = 'LAcounty_assessor.db'

# Introduction

I compute Herfindalh-Hirshmann Indexes (HHIs) for LA County at the block, block group, and tract level. 

Land-use codes are officially detailed [here](https://www.titleadvantage.com/mdocs/LA%20County%20Use%20Codes%20nm.pdf) but I am not about to transcribe manually a pdf file so instead, I use the html table compiled by [dts.edatatrace](http://dts.edatatrace.com/dts3/content/doc/whelp/mergedProjects/dts2tt/mergedProjects/dts2ttcs/land_use_la.htm).

## Steps:

1) I [transfer](#load-assessor-data-and-dump-to-sqlite) the assessor data to an sqlite databse for ease of use. 

2) I [save](#dump-to-geocode) a subset (AIN and lat/lon fields) of the owner data to a csv, make it into a qGIS file.

3) I perform a [spatial join](#join-data-to-census-blocks) of the properties to 2010 Census blocks 

4) I compute [ownership shares](#hirschmann-herfindalh-indexes) and Herfindahl-Hirschmann indexes at the City, Census tract, and Census block level

In [2]:
# turns out, the
#landuse = pd.read_html('http://dts.edatatrace.com/dts3/content/doc/whelp/mergedProjects/dts2tt/mergedProjects/dts2ttcs/land_use_la.htm')[0]
#landuse.columns = landuse.iloc[0]
#landuse = landuse.iloc[1:]

# Load assessor data and dump to sqlite

This saves space and enables me to query the data

In [2]:
# create convenience functions
def getTableNames(dbfile, lookfor="'table'"):
    #open connection
    conn =  sqlite3.connect("/".join([data_path, dbfile]))
    c = conn.cursor()

    #fetch names
    res = c.execute("SELECT name FROM sqlite_master WHERE type=%s;" %lookfor)
    print(res.fetchall())
    
    #close connection
    conn.close()   

def queryDB(query, dbfile):
    conn =  sqlite3.connect("/".join([data_path, dbfile]))
    print("Connection opened to [%s]" %dbfile)
    start_time = time.time()
    df = pd.read_sql_query(query, conn)
    
    print("\tsize of file read in:", df.shape)
    print("\tquery took: %.2fseconds" %(time.time()-start_time))
    conn.close()
    print("\tConnection closed")
    return(df)

In [8]:
## open connection to SQlite database
conn =  sqlite3.connect("/".join([data_path, dbname]))

## create iteratable csv reader
chunksize = 50e4
reader = pd.read_csv("/".join([data_path, 'assessor-owners-20190502.csv']),chunksize=chunksize, low_memory=False)

observationCounter = 0
start_time = time.time() #start timer
tableName = 'assessor'
for i,chunk in enumerate(reader):
    print("Working on chunk num: %d" %i)
    observationCounter+=chunk.shape[0]
    #print("\tNum of rows in chunk: %d" %chunk.shape[0])
    ## fix a few data formats
    chunk.loc[:,'AIN'] = chunk.AIN.astype('float')
    chunk.loc[:,'Units1'] = chunk.Units1.replace("None",0).astype(np.int)
    chunk.loc[:,'UseCode_2'] = chunk.UseCode_2.astype(np.int)
    chunk.loc[:,'MailZip'] = chunk.MailZip.str.replace("-|\s+",'').replace("",0).astype(np.int)
    
    ## dump to database
    chunk.to_sql(tableName,conn, if_exists='append', index=False)
    print("--"*20)
    
print("Total number of observations: %d" %observationCounter)    
print("Done. This took %.2fseconds" %(time.time()-start_time))
print("--"*50)

## repeat above for property history
reader = pd.read_csv("/".join([data_path, 'assessor-owner-change-2009-2019.csv']),chunksize=chunksize, iterator=True, low_memory=False)
tableName = 'history'

observationCounter = 0
start_time = time.time() #start timer
for i,chunk in enumerate(reader):
    print("Working on chunk num: %d" %i)
    #print("\tNum of rows in chunk: %d" %chunk.shape[0])
    observationCounter+=chunk.shape[0]
    chunk.to_sql(tableName,conn, if_exists='append', index=False)
    print("--"*20)
    
print("Total number of observations: %d" %observationCounter)     
print("Done. This took %.2fseconds" %(time.time()-start_time))  
print("--"*50)


conn.close()

Working on chunk num: 0
----------------------------------------
Working on chunk num: 1
----------------------------------------
Working on chunk num: 2
----------------------------------------
Working on chunk num: 3
----------------------------------------
Working on chunk num: 4
----------------------------------------
Total number of observations: 2406966
Done. This took 160.65seconds
----------------------------------------------------------------------------------------------------
Working on chunk num: 0
----------------------------------------
Working on chunk num: 1
----------------------------------------
Working on chunk num: 2
----------------------------------------
Working on chunk num: 3
----------------------------------------
Working on chunk num: 4
----------------------------------------
Working on chunk num: 5
----------------------------------------
Total number of observations: 2679323
Done. This took 48.17seconds
-------------------------------------------------

### Create indexes for faster queries

In [18]:
## creating indexes
conn =  sqlite3.connect("/".join([data_path, dbname]))
c = conn.cursor()
ques =  ["CREATE INDEX usetype ON assessor (UseType);",
    "CREATE INDEX city ON assessor (SitusCity);",
    "CREATE INDEX value ON assessor (Roll_ImpValue);",
    "CREATE INDEX units ON assessor (Units1);",
    "CREATE INDEX ain on assessor (AIN)"]

for i,q in enumerate(ques):
    start_time = time.time()
    print("executing query %d" %i)
    c.execute(q)
    print("\tthis took %.2fseconds" %(time.time()-start_time))
    conn.commit()  

executing query 0
	this took 10.96seconds
executing query 1
	this took 7.19seconds
executing query 2
	this took 9.41seconds
executing query 3
	this took 5.64seconds


In [9]:
conn.close()

In [ ]:
# get table names    
getTableNames(dbname)

# Dump data to geocode

In [18]:
df = queryDB("SELECT AIN, CENTER_LAT, CENTER_LON FROM assessor WHERE UseType = 'Residential'", dbname)
df.to_csv("/".join([data_path, 'toGeocode_lac.csv']))

# Join data to Census Blocks

I used the csv above to create a shapefile in qGIS, now I load that resulting shapefile and the [TIGER shapefile](https://www.census.gov/cgi-bin/geo/shapefiles/index.php) to attach Census blocks to property locations.

In [12]:
import geopandas as gpd
from geopandas.tools import sjoin
print("Modules imported")

point = gpd.GeoDataFrame.from_file("/".join([data_path,'toGeocode_lac.shp'])) # or geojson etc
point.drop('field_1',axis=1,inplace=True)
poly = gpd.GeoDataFrame.from_file("/".join([data_path,'tl_2010_06037_tabblock10.shp']))
print("Shapefiles loaded")

print("Performing spatial join...")
start_time = time.time()
pointInPolys = sjoin(point, poly, how='left')
print("\tSpatial join done. It took: %.2fseconds" %(time.time()-start_time))

pointInPolys.columns

C:\Users\SpiffyApple\Anaconda3\lib\site-packages\geopandas\tools\sjoin.py:44: UserWarning: CRS of frames being joined does not match!
  warn('CRS of frames being joined does not match!')


In [20]:
# save subset to SQLite
pointInPolys[['AIN','STATEFP10', 'COUNTYFP10', 'TRACTCE10', 'BLOCKCE10', 'GEOID10']].to_sql('blocks',conn)

## Create a geocoded view in the sqlite database of the assessor table

In [13]:
conn =  sqlite3.connect("/".join([data_path, dbname]))
c = conn.cursor()

start_time = time.time()
c.execute("CREATE TABLE geoAssessor AS SELECT * FROM assessor INNER JOIN blocks ON assessor.AIN = blocks.AIN;")
print("Done. This query took %.2f" %(time.time()-start_time))
conn.commit()

Done. This query took 116.96


In [9]:
conn.close()

# Hirschmann Herfindalh Indexes

## Load Data

In [3]:
## dont need all of the columns so I only read in the ones that will be required for computing shares and indexes
cols = ['AIN','OwnerName','MailAddress', 'MailCity', 'MailZip','Units1','COUNTYFP10','UseCode_2','TRACTCE10', 'BLOCKCE10',
       'GEOID10','SitusCity']
df = queryDB("SELECT %s FROM geoAssessor;" %(",".join(cols)), dbname)

Connection opened to [LAcounty_assessor.db]
	size of file read in: (2152993, 12)
	query took: 23.02seconds
	Connection closed


In [4]:
df.dtypes

AIN            object
OwnerName      object
MailAddress    object
MailCity       object
MailZip        object
Units1         object
COUNTYFP10     object
UseCode_2      object
TRACTCE10      object
BLOCKCE10      object
GEOID10        object
SitusCity      object
dtype: object

In [5]:
## fix a few column types. For example, units should NOT be object. Not Should AIN?
df.loc[:,'AIN'] = df.AIN.astype('float')
df.loc[:,'Units1'] = df.Units1.replace("None",0).astype(np.int)
df.loc[:,'UseCode_2'] = df.UseCode_2.astype(np.int)
df.loc[:,'MailZip'] = df.MailZip.str.replace("-|\s+",'').replace("",0).astype(np.int)

df.loc[:,'SitusCity'] = df.SitusCity.str.replace(" CA|/CA|-CA|\.|\d+|\([\w\s]+\)",'').str.strip()

In [6]:
print("Number of residential properties: %d" %df.AIN.shape[0])
print("Number of units in LA County: %d" %df.Units1.sum())

Number of residential properties: 2152993
Number of units in LA County: 3078263


In [7]:
## remove units with use code >5
df = df[df.UseCode_2<=5]
print("Number of non-modular or non-mobile residential properties: %d" %df.AIN.shape[0])
print("Number of non-modular or non-mobile in LA County: %d" %df.Units1.sum())

Number of non-modular or non-mobile residential properties: 2149606
Number of non-modular or non-mobile in LA County: 3040543


In [8]:
## drop observations that have 0 units
df.loc[df.Units1==0, 'UseCode_2'].unique()
df = df[df.Units1>0]
print("Number of residential properties with at least 1 unit: %d" %df.AIN.shape[0])
print("Number of residential units in LA County with at least 1 unit: %d" %df.Units1.sum())

Number of residential properties with at least 1 unit: 2034395
Number of residential units in LA County with at least 1 unit: 3040543


In [9]:
## create full mail addresses
df.loc[:,'fullmailaddr'] = df.MailAddress.str.strip().str.replace("  "," ") +", "+df.MailCity.str.strip()

## Mapping Between Owners and Addresses

Do addresses uniquely map to owners? The answer is **no**. A PO Box in Shermak Oaks had 412 names associated with it if we count properties with 0 units. An address in Arcadia has 119 names associated with it if we don't count properties with 0 units. 

What about the other way around, do Owner names uniquely identify addresses? LA City's properties are registered at 47 different addresses. Talking about bureaucracy.

In [10]:
print("Number of unique owners according to addresses: %d" %len(df.fullmailaddr.unique()))
print("Number of unique owners: %d" %len(df.OwnerName.unique()))

Number of unique owners according to addresses: 1683277
Number of unique owners: 1731877


In [11]:
namesPerAddress = df[['fullmailaddr','OwnerName']].drop_duplicates().groupby("fullmailaddr").count().sort_values('OwnerName',ascending=False)

In [12]:
namesPerAddress.head(n=5)

,OwnerName
fullmailaddr,
"610 N SANTA ANITA AVE, ARCADIA CA",119
"8383 WILSHIRE BLVD STE 400, BEVERLY HILLS CA",107
"2016 RIVERSIDE DR, LOS ANGELES CA",100
"PO BOX 48528, LOS ANGELES CA",97
"2351 W 3RD ST, LOS ANGELES CA",95


In [13]:
print("Names associated with the Arcadia address:")
df.loc[df.fullmailaddr=='610 N SANTA ANITA AVE, ARCADIA CA','OwnerName'].drop_duplicates().head()

Names associated with the Arcadia address:


660             YALAMANCHILIRAO R
21136    PI PROPERTIES NO 128 LLC
26046     PI PROPERTIES NO 79 LLC
35127       PI PROPERTIES 118 LLC
40468          PI CONDOMINIUMS LP
Name: OwnerName, dtype: object

In [14]:
addrPerName = df[['fullmailaddr','OwnerName']].drop_duplicates().groupby("OwnerName").count().sort_values('fullmailaddr',ascending=False)

In [15]:
addrPerName.head(n=5)

,fullmailaddr
OwnerName,
L A CITY,47
GONZALEZJOSE,45
EQUITY TRUST COMPANY CSTDN,44
GARCIAMARIA,42
GONZALEZJOSE A,38


In [16]:
print("Addresses associated with LA City:")
df.loc[df.OwnerName == 'L A CITY', 'fullmailaddr'].drop_duplicates().head(n=10)

Addresses associated with LA City:


480             111 E 1ST ST  STE 212, LOS ANGELES CA
849       6053 W CENTURY BLVD  4TH FL, LOS ANGELES CA
1894              1 WORLD WAY  8TH FL, LOS ANGELES CA
97849                   314 VIEWLAND PL, SAN PEDRO CA
299961            425 S PALOS VERDES ST, SAN PEDRO CA
345969         200 N MAIN ST  RM 1330, LOS ANGELES CA
349763    425 S PALOS VERDES ST  5  FLR, SAN PEDRO CA
354635                     424 W 7TH ST, SAN PEDRO CA
413818          1200 W 7TH ST  6TH FL, LOS ANGELES CA
443390           111 E 1ST ST  RM 213, LOS ANGELES CA
Name: fullmailaddr, dtype: object

## County level HHI

At the county level, the housing market looks virtually perfectly competitive

In [17]:
cntylvl = df.loc[:,['Units1','fullmailaddr']].groupby('fullmailaddr').sum()
cntylvl.loc[:,'share'] = (cntylvl.Units1/cntylvl.Units1.sum()).round(4)

In [18]:
print("Largest property holders in LA County by owner address:")
cntylvl.sort_values('Units1',ascending=False).iloc[:5]

Largest property holders in LA County by owner address:


,Units1,share
fullmailaddr,,
"2016 RIVERSIDE DR, LOS ANGELES CA",7326,0.0024
"PO BOX 87407, CHICAGO IL",7254,0.0024
"PO BOX 59365, SCHAUMBURG IL",6516,0.0021
"9441 WILSHIRE BLVD # PH, BEVERLY HILLS CA",6452,0.0021
"5150 OVERLAND AVE, CULVER CITY CA",5296,0.0017


In [19]:
print("County level HHI %.3f" %np.sum(np.square(cntylvl.share)))

County level HHI 0.000


## City Level HHI

Alas, the city level proves rather bad mostly because entries for the location of the property contain countless typographical errors. In fact, if we treat unique entries for the SituCity as the number of cities in LA County then there are 637 cities, including famous locations such as CALABASSAS, RLLNG HLS, LOS ANGELESA, and the ever-a-desired-destination AGUA DUICE. 

I could hire someone do correct the misspellings but it probably isn't worth it.

In [20]:
sharesDict = {}
hhiDict = {}

In [21]:
lvl = 'SitusCity'
sharesDict[lvl] = df.loc[:,['Units1','fullmailaddr',lvl]].groupby([lvl,'fullmailaddr']).sum()

In [22]:
hhiDict[lvl] = np.square(np.divide(sharesDict[lvl].Units1,sharesDict[lvl].groupby(lvl).transform('sum').Units1)).groupby(lvl).sum()
hhiDict[lvl].name = 'HHI'

In [23]:
# select cities:
cities = ['LOS ANGELES','PASADENA' ,'CULVER CITY', 'SANTA MONICA','WEST HOLLYWOOD','WRIGHTWOOD']
hhiDict[lvl][cities]

SitusCity
LOS ANGELES       0.000171
PASADENA          0.000564
CULVER CITY       0.000415
SANTA MONICA      0.001341
WEST HOLLYWOOD    0.001948
WRIGHTWOOD        0.011529
Name: HHI, dtype: float64

## Tract and Block Level HHI

In [33]:
df.loc[:,'tractGEOID'] = '06'+df.COUNTYFP10+df.TRACTCE10
sub = df.loc[(df.UseCode_2 > 1)]

In [34]:
levels = ['tractGEOID', 'GEOID10']
for lvl in levels:
    sharesDict[lvl] = df.loc[:,['Units1','fullmailaddr',lvl]].groupby([lvl,'fullmailaddr']).sum()
    hhiDict[lvl] = pd.DataFrame(np.square(np.divide(sharesDict[lvl].Units1,sharesDict[lvl].groupby(lvl).transform('sum').Units1)).groupby(lvl).sum())
    
    sharesDict[lvl] = sub.loc[:,['Units1','fullmailaddr',lvl]].groupby([lvl,'fullmailaddr']).sum()
    hhiDict[lvl].loc[:,'income'] = np.square(np.divide(sharesDict[lvl].Units1,sharesDict[lvl].groupby(lvl).transform('sum').Units1)).groupby(lvl).sum()
    
    hhiDict[lvl].columns = ['allProperties','incomeProperties']

In [36]:
## Save Tract and Block results to CSVs
hhiDict['tractGEOID'].to_csv("/".join([data_path, 'tractHHI_LA.csv']), header=True)
hhiDict['GEOID10'].to_csv("/".join([data_path, 'blockHHI_LA.csv']),header=True)